In [2]:
import pandas as pd
import numpy as np
import re
final_data = pd.read_csv('final_data.csv')
battle_result = pd.read_csv('battle_result.csv')


In [3]:
final_data

#print the highest and lowest trophy count


,event_map,battle_type,mode,team 1 brawler 1,team 1 brawler 1 power,team 1 brawler 1 trophies,team 1 brawler 2,team 1 brawler 2 power,team 1 brawler 2 trophies,team 1 brawler 3,...,team 1 brawler 3 trophies,team 2 brawler 1,team 2 brawler 1 power,team 2 brawler 1 trophies,team 2 brawler 2,team 2 brawler 2 power,team 2 brawler 2 trophies,team 2 brawler 3,team 2 brawler 3 power,team 2 brawler 3 trophies
0,Deep Diner,ranked,gemGrab,GUS,11.0,717.0,CROW,11.0,791.0,DARRYL,...,701.0,MORTIS,11.0,791.0,FANG,11.0,740.0,JESSIE,10.0,682.0
1,Sneaky Fields,ranked,brawlBall,RICO,11.0,782.0,NITA,11.0,837.0,EL PRIMO,...,890.0,MORTIS,11.0,799.0,EMZ,10.0,784.0,FRANK,11.0,905.0
2,Sneaky Fields,ranked,brawlBall,POCO,11.0,673.0,JACKY,10.0,681.0,SPIKE,...,704.0,GALE,11.0,651.0,SURGE,11.0,649.0,DARRYL,11.0,752.0
3,Sneaky Fields,ranked,brawlBall,LEON,10.0,641.0,SANDY,10.0,604.0,BIBI,...,663.0,BIBI,11.0,654.0,JACKY,10.0,625.0,CROW,11.0,674.0
4,Sneaky Fields,ranked,brawlBall,COLETTE,7.0,403.0,BUSTER,11.0,419.0,GENE,...,411.0,MAX,8.0,395.0,COLETTE,7.0,390.0,FANG,7.0,446.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95476,Flaring Phoenix,ranked,knockout,BONNIE,11.0,450.0,TICK,9.0,500.0,CHESTER,...,558.0,GRAY,9.0,483.0,COLT,10.0,508.0,EDGAR,9.0,478.0
95477,Minecart Madness,ranked,gemGrab,NANI,11.0,497.0,BROCK,9.0,516.0,COLT,...,500.0,BYRON,9.0,485.0,BO,10.0,520.0,BROCK,10.0,514.0
95478,Beach Ball,ranked,brawlBall,TICK,9.0,494.0,RUFFS,10.0,445.0,SQUEAK,...,494.0,SURGE,9.0,482.0,AMBER,9.0,492.0,SPROUT,9.0,492.0
95479,Beach Ball,ranked,brawlBall,ASH,10.0,520.0,DYNAMIKE,9.0,497.0,BARLEY,...,525.0,LOU,11.0,530.0,RICO,10.0,508.0,TICK,9.0,500.0


In [4]:
# Columns to be linked
linked_columns = [
    'team 1 brawler 1', 'team 1 brawler 2', 'team 1 brawler 3',
    'team 2 brawler 1', 'team 2 brawler 2', 'team 2 brawler 3'
]
# Dictionary to store the mappings
mappings = {}

# Concatenate the linked columns into a single series
concatenated = pd.concat([final_data[col] for col in linked_columns])

# Factorize the concatenated series
factorized_values, unique_values = pd.factorize(concatenated)

# Store the mappings for the linked columns
mappings['Brawlers'] = dict(enumerate(unique_values))

# Apply the factorized values back to the original linked columns
for i, col in enumerate(linked_columns):
    final_data[col] = factorized_values[i * len(final_data):(i + 1) * len(final_data)]

# Enumerate values in each remaining text column and store the mappings
for column in final_data.columns:
    if column not in linked_columns and final_data[column].dtype == 'object':
        final_data[column], unique_values = pd.factorize(final_data[column])
        mappings[column] = dict(enumerate(unique_values))

# Print the first value in the first column
print(final_data.iloc[0, 0])

# Print the mappings
print(mappings)

0
{'Brawlers': {0: 'GUS', 1: 'RICO', 2: 'POCO', 3: 'LEON', 4: 'COLETTE', 5: 'ROSA', 6: 'TICK', 7: 'JANET', 8: 'MR. P', 9: 'NITA', 10: 'EL PRIMO', 11: 'FANG', 12: 'BONNIE', 13: 'MANDY', 14: 'PIPER', 15: 'SPIKE', 16: '8-BIT', 17: 'JESSIE', 18: 'BEA', 19: 'DYNAMIKE', 20: 'BARLEY', 21: 'GRAY', 22: 'ASH', 23: 'STU', 24: 'COLT', 25: 'DARRYL', 26: 'MORTIS', 27: 'FRANK', 28: 'BIBI', 29: 'GRIFF', 30: 'SQUEAK', 31: 'CHESTER', 32: 'GENE', 33: 'MAX', 34: 'JACKY', 35: 'OTIS', 36: 'AMBER', 37: 'EMZ', 38: 'LOU', 39: 'SANDY', 40: 'BUSTER', 41: 'RUFFS', 42: 'PAM', 43: 'GROM', 44: 'BROCK', 45: 'BUZZ', 46: 'SURGE', 47: 'NANI', 48: 'CROW', 49: 'BULL', 50: 'TARA', 51: 'BYRON', 52: 'SAM', 53: 'SHELLY', 54: 'GALE', 55: 'BO', 56: 'LOLA', 57: 'CARL', 58: 'PENNY', 59: 'EDGAR', 60: 'BELLE', 61: 'SPROUT', 62: 'MEG', 63: 'EVE'}, 'event_map': {0: 'Deep Diner', 1: 'Sneaky Fields', 2: 'Flaring Phoenix', 3: 'Pinhole Punt', 4: 'Double Swoosh', 5: 'Open Zone', 6: 'Temple Ruins', 7: 'Super Beach', 8: 'Crystal Arcade', 9:

In [5]:
# List of columns to check for -1.0 values
columns_to_check = [
    'team 1 brawler 1 power', 'team 1 brawler 1 trophies',
    'team 1 brawler 2 power', 'team 1 brawler 2 trophies',
    'team 1 brawler 3 power', 'team 1 brawler 3 trophies',
    'team 2 brawler 1 power', 'team 2 brawler 1 trophies',
    'team 2 brawler 2 power', 'team 2 brawler 2 trophies',
    'team 2 brawler 3 power', 'team 2 brawler 3 trophies'
]


# Identify rows with -1.0 values in the specified columns
rows_to_remove = final_data[columns_to_check].isin([-1.0]).any(axis=1)

# Remove these rows from final_data
final_data= final_data[~rows_to_remove]

# Remove the same rows from battle_result
battle_result = battle_result[~rows_to_remove]


In [6]:
assert len(final_data) == len(battle_result), "DataFrames have different number of rows after cleaning."


In [7]:
# Factorize the values in the column
battle_result['battle_result'], unique_values = pd.factorize(battle_result['battle_result'])

# Print the enumerated battle_result DataFrame
print(battle_result)

# Check how many distinct values there are in the column
distinct_values_count = battle_result['battle_result'].nunique()

# Print the number of distinct values in the column
print(f"\nNumber of distinct values in 'result': {distinct_values_count}")

print("number of 1s in battle result",battle_result['battle_result'].value_counts()[1])
print("number of 0s in battle result",battle_result['battle_result'].value_counts()[0])

       battle_result
0                  0
1                  0
2                  0
3                  0
4                  1
...              ...
95476              0
95477              1
95478              0
95479              0
95480              1

[95367 rows x 1 columns]

Number of distinct values in 'result': 2
number of 1s in battle result 55817
number of 0s in battle result 39550


In [8]:
# Add a unique identifier to preserve alignment between battle_result and final_data
battle_result['original_index'] = battle_result.index
final_data['original_index'] = final_data.index

# Separate rows with 1 and 0 in battle_result
ones = battle_result[battle_result['battle_result'] == 1]
zeros = battle_result[battle_result['battle_result'] == 0]

# Get the count of rows with 0
num_zeros = len(zeros)

# Randomly sample rows with 1 to match the count of rows with 0
ones_sampled = ones.sample(n=num_zeros, random_state=42)

# Combine the sampled rows with 0 rows
battle_result = pd.concat([zeros, ones_sampled])

# Use the 'original_index' to filter final_data
final_data = final_data[final_data['original_index'].isin(battle_result['original_index'])]

# Drop the 'original_index' column from both DataFrames
battle_result = battle_result.drop(columns=['original_index']).reset_index(drop=True)
final_data = final_data.drop(columns=['original_index']).reset_index(drop=True)

# Print or save the balanced final_data
battle_result

C:\Users\ellio\AppData\Local\Temp\ipykernel_26104\3508247976.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['original_index'] = final_data.index


,battle_result
0,0
1,0
2,0
3,0
4,0
...,...
79095,1
79096,1
79097,1
79098,1


In [8]:
assert len(final_data) == len(battle_result), "DataFrames have different number of rows after cleaning."


In [9]:
print("number of 1s in battle result",battle_result['battle_result'].value_counts()[1])
print("number of 0s in battle result",battle_result['battle_result'].value_counts()[0])

number of 1s in battle result 39550
number of 0s in battle result 39550


In [10]:
from sklearn.preprocessing import normalize

# Select the trophy columns
trophy_columns = [col for col in final_data.columns if 'trophies' in col]

# Apply L2 normalization and update the DataFrame in place
final_data[trophy_columns] = normalize(final_data[trophy_columns], norm='l2', axis=0)

# Verify the normalization
print("Highest trophy count after normalization: ", final_data['team 2 brawler 3 trophies'].max())
print("Lowest trophy count after normalization: ", final_data['team 2 brawler 1 trophies'].min())

Highest trophy count after normalization:  0.010114327536164992
Lowest trophy count after normalization:  0.0


In [11]:
print("Highest trophy count: ", final_data['team 2 brawler 3 trophies'].max())
print("Lowest trophy count: ", final_data['team 2 brawler 1 trophies'].min())

Highest trophy count:  0.010114327536164992
Lowest trophy count:  0.0


In [12]:
from sklearn.preprocessing import MinMaxScaler

# Select the power level columns
power_level_columns = [col for col in final_data.columns if 'power' in col]

# Apply Min-Max Scaling
scaler = MinMaxScaler()
final_data[power_level_columns] = scaler.fit_transform(final_data[power_level_columns])

# Verify the scaling
print(final_data[power_level_columns].head())

   team 1 brawler 1 power  team 1 brawler 2 power  team 1 brawler 3 power  \
0                     1.0                     1.0                     1.0   
1                     1.0                     1.0                     1.0   
2                     1.0                     0.9                     0.9   
3                     0.9                     0.9                     1.0   
4                     0.6                     1.0                     0.6   

   team 2 brawler 1 power  team 2 brawler 2 power  team 2 brawler 3 power  
0                     1.0                     1.0                     0.9  
1                     1.0                     0.9                     1.0  
2                     1.0                     1.0                     1.0  
3                     1.0                     0.9                     1.0  
4                     0.7                     0.6                     0.6  


In [13]:
#print lowest and highest power level
print("Highest power level: ", final_data['team 2 brawler 3 power'].max())
print("Lowest power level: ", final_data['team 2 brawler 1 power'].min())

Highest power level:  1.0
Lowest power level:  0.0


In [ ]:
#One-hot encoding

# Specify the columns to one-hot encode
columns_to_encode = ['team 1 brawler 1', 'team 1 brawler 2 ', 'team 1 brawler 3', 'team 2 brawler 1', 'team 2 brawler 2', 'team 2 brawler 3', 'event_map', 'mode']

final_data = pd.get_dummies(final_data, columns=columns_to_encode)

print("DataFrame with One-Hot Encoding:")
print(final_data.head())


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_data, battle_result, test_size=0.2, random_state=42)
X_test, X_eval, y_test, y_eval = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

assert len(X_test) == len(y_test), "DataFrames have different number of rows after cleaning."

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
y_eval = y_eval.values.ravel()

In [19]:
X_test

,event_map,battle_type,mode,team 1 brawler 1,team 1 brawler 1 power,team 1 brawler 1 trophies,team 1 brawler 2,team 1 brawler 2 power,team 1 brawler 2 trophies,team 1 brawler 3,...,team 1 brawler 3 trophies,team 2 brawler 1,team 2 brawler 1 power,team 2 brawler 1 trophies,team 2 brawler 2,team 2 brawler 2 power,team 2 brawler 2 trophies,team 2 brawler 3,team 2 brawler 3 power,team 2 brawler 3 trophies
58367,14,1,5,47,0.7,0.003054,26,0.7,0.003010,53,...,0.003016,5,0.8,0.003053,53,0.8,0.003059,10,0.8,0.002964
1165,34,0,3,49,1.0,0.005278,27,1.0,0.005009,10,...,0.004862,37,1.0,0.004911,25,1.0,0.005107,15,1.0,0.004211
43521,6,0,4,59,0.8,0.004412,27,0.8,0.004533,26,...,0.004654,40,0.8,0.004238,43,0.8,0.004300,9,0.9,0.004157
16264,29,0,4,44,0.9,0.000018,12,0.9,0.000048,43,...,0.000059,11,1.0,0.000042,55,1.0,0.000060,47,1.0,0.000053
69690,1,0,1,50,1.0,0.004833,45,0.9,0.004894,6,...,0.004464,26,1.0,0.005001,34,1.0,0.004824,18,1.0,0.004781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36535,20,1,2,33,1.0,0.000072,37,0.9,0.000072,48,...,0.000065,46,1.0,0.000072,2,0.9,0.000072,39,0.9,0.000071
64302,15,2,3,2,0.9,0.005717,22,1.0,0.005346,29,...,0.005581,22,1.0,0.004737,26,1.0,0.004240,6,1.0,0.004353
26868,3,0,1,19,0.8,0.003054,31,1.0,0.003209,60,...,0.003188,15,0.8,0.003053,12,0.8,0.003144,43,0.8,0.003195
57403,22,0,4,61,0.8,0.002982,24,0.7,0.002926,51,...,0.002452,58,0.7,0.002753,0,0.8,0.002812,57,0.8,0.002702


In [21]:
import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the model
model = MultinomialNB()

# Define the hyperparameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]  # Different values for the smoothing parameter
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Measure the time taken to train the model
start_train_time = time.time()
grid_search.fit(X_train, y_train)
end_train_time = time.time()

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")
print(f"Time taken to train the model: {end_train_time - start_train_time:.2f} seconds")

# Measure the time taken to generate predictions
start_predict_time = time.time()
y_pred_test = best_model.predict(X_test)
end_predict_time = time.time()

accuracy_eval = accuracy_score(y_test, y_pred_test)
print(f"Accuracy with Best Parameters: {accuracy_eval}")
print(f"Time taken to generate predictions: {end_predict_time - start_predict_time:.2f} seconds")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best Parameters: {'alpha': 0.1}
Time taken to train the model: 0.97 seconds
Accuracy with Best Parameters: 0.5061946902654867
Time taken to generate predictions: 0.00 seconds


In [22]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the model
logreg = LogisticRegression(random_state=16)

# Define the hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2'],       # Regularization type
    'solver': ['liblinear', 'saga'],  # Solvers that support 'l1' and 'l2'
    'max_iter': [100, 200, 500]    # Number of iterations
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Measure the time taken to train the model
start_train_time = time.time()
grid_search.fit(X_train, y_train)
end_train_time = time.time()

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")
print(f"Time taken to train the model: {end_train_time - start_train_time:.2f} seconds")

# Measure the time taken to generate predictions
start_predict_time = time.time()
y_pred = best_model.predict(X_test)
end_predict_time = time.time()

accuracy_eval = accuracy_score(y_test, y_pred)
print(f"Accuracy with Best Parameters: {accuracy_eval}")
print(f"Time taken to generate predictions: {end_predict_time - start_predict_time:.2f} seconds")

Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the 

Best Parameters: {'C': 0.01, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}
Time taken to train the model: 1120.43 seconds
Accuracy with Best Parameters: 0.5233881163084703
Time taken to generate predictions: 0.00 seconds


In [23]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the model
model = RandomForestClassifier(random_state=16)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],          # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],         # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],         # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],           # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]               # Whether bootstrap samples are used when building trees
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Measure the time taken to train the model
start_train_time = time.time()
grid_search.fit(X_train, y_train)
end_train_time = time.time()

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")
print(f"Time taken to train the model: {end_train_time - start_train_time:.2f} seconds")

# Measure the time taken to generate predictions
start_predict_time = time.time()
y_pred = best_model.predict(X_test)
end_predict_time = time.time()

# Evaluate the model
accuracy_eval = accuracy_score(y_test, y_pred)
print(f"Accuracy with Best Parameters: {accuracy_eval}")
print(f"Time taken to generate predictions: {end_predict_time - start_predict_time:.2f} seconds")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Time taken to train the model: 31155.76 seconds
Accuracy with Best Parameters: 0.5988621997471555
Time taken to generate predictions: 0.57 seconds


In [ ]:
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the model
model = MLPClassifier(random_state=16)

# Define the hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Number of neurons in hidden layers
    'activation': ['relu', 'tanh', 'logistic'],                 # Activation functions
    'solver': ['adam', 'sgd'],                                  # Optimization solvers
    'alpha': [0.0001, 0.001, 0.01],                             # L2 regularization parameter
    'learning_rate': ['constant', 'adaptive'],                  # Learning rate schedule
    'max_iter': [200, 500]                                      # Maximum number of iterations
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Measure the time taken to train the model
start_train_time = time.time()
grid_search.fit(X_train, y_train)
end_train_time = time.time()

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")
print(f"Time taken to train the model: {end_train_time - start_train_time:.2f} seconds")

# Measure the time taken to generate predictions
start_predict_time = time.time()
y_pred = best_model.predict(X_test)
end_predict_time = time.time()

# Evaluate the model
accuracy_eval = accuracy_score(y_test, y_pred)
print(f"Accuracy with Best Parameters: {accuracy_eval}")
print(f"Time taken to generate predictions: {end_predict_time - start_predict_time:.2f} seconds")

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

In [16]:
num_unique_values = final_data.iloc[:, 3].nunique()
print(f"Number of different values in the fourth column: {num_unique_values}")

Number of different values in the fourth column: 64


In [3]:
num_nan_event_map = final_data['mode'].isna().sum()
print(f"\nNumber of NaN values in 'event_map': {num_nan_event_map}")


Number of NaN values in 'event_map': 0


In [3]:
import re

# The input string
input_string = "[[{'tag': '#LGC0PQ98', 'name': '🍁PassØut™', 'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 14}}, {'tag': '#VULUQUY8', 'name': '金|𝔻ανσχ💔', 'brawler': {'id': 16000019, 'name': 'PENNY', 'power': 11, 'trophies': 16}}, {'tag': '#92CRYR8JR', 'name': 'Adͥℝiͣaͫn🌎⚡', 'brawler': {'id': 16000054, 'name': 'FANG', 'power': 11, 'trophies': 16}}], [{'tag': '#PJJJR9JQ2', 'name': 'schwule gans', 'brawler': {'id': 16000015, 'name': 'PIPER', 'power': 11, 'trophies': 12}}, {'tag': '#YVQQJY0', 'name': 'мєℓιн™', 'brawler': {'id': 16000025, 'name': 'CARL', 'power': 11, 'trophies': 15}}, {'tag': '#9GYCJYGP', 'name': 'Armo', 'brawler': {'id': 16000064, 'name': 'GRAY', 'power': 11, 'trophies': 15}}]]"

# Regular expression to match the desired substrings
pattern = r"'brawler': \{[^}]+\}"

# Find all matches
matches = re.findall(pattern, input_string)

# Store matches in a list
brawler_list = matches

# Print the list
for brawler in brawler_list:
    print(brawler)

'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 14}
'brawler': {'id': 16000019, 'name': 'PENNY', 'power': 11, 'trophies': 16}
'brawler': {'id': 16000054, 'name': 'FANG', 'power': 11, 'trophies': 16}
'brawler': {'id': 16000015, 'name': 'PIPER', 'power': 11, 'trophies': 12}
'brawler': {'id': 16000025, 'name': 'CARL', 'power': 11, 'trophies': 15}
'brawler': {'id': 16000064, 'name': 'GRAY', 'power': 11, 'trophies': 15}
